## **AI Programming - Final Project**
### Movie Recommendation System

Sources:
Source 1: https://www.tensorflow.org/recommenders/examples/basic_retrieval

Source 2: https://blog.paperspace.com/movie-recommender-tensorflow/

Source 3: https://medium.com/@bilalsedef/creating-movie-recommender-system-with-tensorflow-recommenders-tfrs-acc19fd9f7a0

In [1]:
# Installing the libraries that we will need to use

! pip install -q tensorflow-recommenders
! pip install -q --upgrade tensorflow-datasets
! pip install -q scann

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 99.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 108.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 KB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 114.4 MB/s eta 0:00:00


In [2]:
# Importing the libraries that we will need to use

import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [3]:
# Load the data and split it into training and test set

# Ratings data
ratings = tfds.load("movielens/1m-ratings", split="train")

# Movies data
movies = tfds.load("movielens/1m-movies", split="train")

# We only use the ratings data & within that, we use the movie title & user id

ratings = ratings.map(lambda x: { "movie_title": x["movie_title"],
    "user_id": x["user_id"],})
movies = movies.map(lambda x: x["movie_title"])

# We split the data in 80% training and 20% testing sets

tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

# Extract unique sets of movie titles and user IDs

movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

...

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/1m-movies/0.1.1.incompleteUZKVC5/movielens-train.tfrecord*...:  …

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/1m-movies/0.1.1. Subsequent calls will reuse this data.


In [4]:
# Build the query tower (Set up the retrieval model)

embedding_dimension = 32

user_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)])

# Build the movie candidate tower 

movie_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_movie_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, 
                            embedding_dimension)])

#Evaluate the performance of the model 

metrics = tfrs.metrics.FactorizedTopK(candidates=movies.batch(128).
                                      map(movie_model))

# Evaluate the loss used to train the model

task = tfrs.tasks.Retrieval(metrics=metrics)

In [5]:
# Put it together into a model

class MovielensModel(tfrs.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], 
                   training=False) -> tf.Tensor:
# We pick out the user features and pass them into the user model
    user_embeddings = self.user_model(features["user_id"])
# And pick out the movie features & pass them into the movie model, 
# getting embeddings back
    positive_movie_embeddings = self.movie_model(features["movie_title"])

# The task computes the loss and the metrics
    return self.task(user_embeddings, positive_movie_embeddings)

In [6]:
# Fit and evaluate the model

model = MovielensModel(user_model, movie_model)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1))
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()
model.fit(cached_train, epochs=10)

Epoch 1/10
10/10 [==============================] - 41s 2s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 2.8750e-04 - factorized_top_k/top_10_categorical_accuracy: 0.0010 - factorized_top_k/top_50_categorical_accuracy: 0.0077 - factorized_top_k/top_100_categorical_accuracy: 0.0186 - loss: 70701.1754 - regularization_loss: 0.0000e+00 - total_loss: 70701.1754
Epoch 2/10
10/10 [==============================] - 24s 2s/step - factorized_top_k/top_1_categorical_accuracy: 1.2500e-05 - factorized_top_k/top_5_categorical_accuracy: 0.0259 - factorized_top_k/top_10_categorical_accuracy: 0.0466 - factorized_top_k/top_50_categorical_accuracy: 0.1409 - factorized_top_k/top_100_categorical_accuracy: 0.2161 - loss: 65357.3423 - regularization_loss: 0.0000e+00 - total_loss: 65357.3423
Epoch 3/10
10/10 [==============================] - 25s 2s/step - factorized_top_k/top_1_categorical_accuracy: 0.0034 - factorized_top_k/top_5_categorical_a

In [7]:
# Evaluate the model on the test set

model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 21s 2s/step - factorized_top_k/top_1_categorical_accuracy: 5.0000e-05 - factorized_top_k/top_5_categorical_accuracy: 0.0011 - factorized_top_k/top_10_categorical_accuracy: 0.0028 - factorized_top_k/top_50_categorical_accuracy: 0.0259 - factorized_top_k/top_100_categorical_accuracy: 0.0558 - loss: 45191.1680 - regularization_loss: 0.0000e+00 - total_loss: 45191.1680


{'factorized_top_k/top_1_categorical_accuracy': 4.999999873689376e-05,
 'factorized_top_k/top_5_categorical_accuracy': 0.0010999999940395355,
 'factorized_top_k/top_10_categorical_accuracy': 0.00279999990016222,
 'factorized_top_k/top_50_categorical_accuracy': 0.02590000070631504,
 'factorized_top_k/top_100_categorical_accuracy': 0.05575000122189522,
 'loss': 41408.734375,
 'regularization_loss': 0,
 'total_loss': 41408.734375}

In [8]:
# We can make predictions now

index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(tf.data.Dataset.zip((movies.batch(100), 
                                              movies.batch(100).map(model.movie_model))))

# Get recommendations

_, titles = index(tf.constant(["20"]))
print(f"Recommendations for user 20: {titles[0, :3]}")

_, titles = index(tf.constant(["68"]))
print(f"Recommendations for user 68: {titles[0, :3]}")

_, titles = index(tf.constant(["99"]))
print(f"Recommendations for user 99: {titles[0, :3]}")

_, titles = index(tf.constant(["129"]))
print(f"Recommendations for user 129: {titles[0, :3]}")

_, titles = index(tf.constant(["212"]))
print(f"Recommendations for user 212: {titles[0, :3]}")

Recommendations for user 20: [b'Matrix, The (1999)' b'Saving Private Ryan (1998)'
 b'Iron Giant, The (1999)']
Recommendations for user 68: [b'Radio Days (1987)' b'Persuasion (1995)' b'Jungle Book, The (1967)']
Recommendations for user 99: [b'Next Best Thing, The (2000)' b'Sixteen Candles (1984)' b'X-Men (2000)']
Recommendations for user 129: [b'True Crime (1999)' b"On Her Majesty's Secret Service (1969)"
 b'True Crime (1995)']
Recommendations for user 212: [b'Forget Paris (1995)' b'Dumb & Dumber (1994)' b'Screwed (2000)']
